Import packages

In [1]:
import requests
from bs4 import BeautifulSoup
import os

Webscraping from Wikipedia and storing info in a folder

In [ ]:
import requests
from bs4 import BeautifulSoup
import os

def fetch_wikipedia_html(title):
    # Fetch the HTML content of a Wikipedia page using the Wikipedia API
    URL = f"https://en.wikipedia.org/w/api.php"
    PARAMS = {
        "action": "parse",
        "page": title,
        "format": "json",
        "prop": "text",
        "redirects": True
    }
    
    response = requests.get(URL, params=PARAMS)
    response.raise_for_status()  # Raise an exception for HTTP errors
    
    data = response.json()
    return data['parse']['text']['*']

def save_html_to_file(html_content, filename):
    with open(filename, "w", encoding="utf-8") as file:
        file.write(html_content)

def fetch_and_save_article(title, base_dir):
    # Fetch and save the main article
    print(f"Fetching article: {title}")
    html_content = fetch_wikipedia_html(title)
    
    # Use BeautifulSoup to prettify the HTML and find hyperlinks
    soup = BeautifulSoup(html_content, "html.parser")
    pretty_html = soup.prettify()
    
    main_filename = os.path.join(base_dir, title.replace(" ", "_") + ".html")
    save_html_to_file(pretty_html, main_filename)
    print(f"Saved {title} as {main_filename}")
    
    # Fetch and save each hyperlink
    for link in soup.find_all('a', href=True):
        href = link['href']
        if href.startswith('/wiki/') and not ':' in href:
            linked_title = href.split('/wiki/')[1]
            linked_title = linked_title.replace('_', ' ')
            # Skip identifier links
            if any(identifier in linked_title.lower() for identifier in ["doi", "pmid", "isbn", "bibcode", "s2cid", "hdl", "arxiv", "pmc"]):
                continue
            try:
                linked_html = fetch_wikipedia_html(linked_title)
                linked_soup = BeautifulSoup(linked_html, "html.parser")
                linked_pretty_html = linked_soup.prettify()
                
                linked_filename = os.path.join(base_dir, linked_title.replace(" ", "_") + ".html")
                save_html_to_file(linked_pretty_html, linked_filename)
                print(f"Saved linked article {linked_title} as {linked_filename}")
            except Exception as e:
                print(f"Failed to fetch/save linked article {linked_title}: {e}")

# Directory to save the articles
base_dir = "wikiArticles"
if not os.path.exists(base_dir):
    os.makedirs(base_dir)

# Main article to fetch and save
main_article = "Temporal envelope and fine structure"
fetch_and_save_article(main_article, base_dir)